In [1]:
from aleph_alpha_client import Client, Prompt, CompletionRequest, CompletionResponse
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv

from langchain.llms import AlephAlpha

### Step 0: Define some helper functions

In [2]:
load_dotenv()

# for using the Aleph Alpha API
client = Client(token=os.getenv("AA_TOKEN"))

# for using LangChain
aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), model="luminous-extended")

#### Step 1: Use the completion function of the API


In [8]:
# using the full-fledged AA API for a simple completion

# Define the prompt
prompt = Prompt.from_text("""### Instructions: Write a poem about Hedgehogs.
                          
                          ### Response:""")

# Create a completion request with parameters
request = CompletionRequest(prompt=prompt, 
                            maximum_tokens=100, 
                            temperature=0.0, 
                            stop_sequences=[])

# Send the request to the API
response = client.complete(request=request, model="luminous-base-control-beta")

# Get the completion
completion = response.completions[0].completion
print(completion)

 Hedgehog, hedgehog, where have you been?
 I've been looking everywhere for you.
 You were last seen in the garden,
 Sitting on a sunny stone.
 You were so happy there,
 You didn't want to go.
 But now you


In [9]:
# using LangChain for a simple completion

# define the prompt
prompt = "An apple a day"

# define the parameters
params = {
    "temperature": 0.5,
    "model": "luminous-extended",
    "maximum_tokens": 100
}

# define the model
aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), **params)

# get the completion
response = aleph_alpha(prompt=prompt, stop=["\n"])
print(response)

 keeps the doctor away, but what about the doctor who keeps you?


#### Step 2: Write a few shot prompt that generates keywords

##### Let's import some data to work on

In [10]:
# Read the data in the data.md file
with open("data.md", "r") as f:
    data = f.read()
    
# Split the data into a list of texts
texts = data.split("#####")

print(f"data: {data[:100]}")
print(f"texts: {texts[10][:100]}")

data: SUMMARY

# Most OECD countries have developed and implemented national sustainable development strat
texts:  INDICATORS AND TARGETS

The development and incorporation of quantitative indicators can help remov


In [19]:
print(texts[1])

 INTRODUCTION

# Background

Governments first agreed to prepare national sustainable development strategies as part of *Agenda 21,* signed at the United Nations Conference on Environment and Development (the Rio Earth Summit) in 1992. The purpose of these strategies was to translate the Summit’s ideas and commitments into concrete policies and actions. Governments agreed to “adopt national strategies for sustainable development [which should] build upon and harmonise the various sectoral, economic, social and environmental policies and plans that are operating in the country. Its goals should be to ensure socially responsible economic development for the benefit of future generations”.

In 1997, the Special Session of the UN General Assembly met to review progress since the Rio Summit and set a target date of 2002 for introducing national sustainable development strategies. Subsequently, the 2002 World Summit on Sustainable Development (WSSD) noted the lack of progress in strategy dev

###### Let's create Keywords for the first 10 rows of the dataset

In [11]:
# Get the first 10 texts
texts_for_keywords = texts

# TODO write a function that takes a text and returns a list of keywords
def get_keywords(text):
    # TODO Write a prompt that generates keywords for any text
    # Tipp: Use few-shot learning
    
    # 1. define the prompt
    prompt = Prompt.from_text(f"""Identify matching keywords for each text.
###
Text: The "Whiskey War" is an ongoing conflict between Denmark and Canada over ownership of Hans Island. The dispute began in 1973, when Denmark and Canada reached an agreement on Greenland's borders. However, no settlement regarding Hans Island could be reached by the time the treaty was signed. Since then both countries have used peaceful means - such as planting their national flag or burying liquor - to draw attention to the disagreement.
Keywords: Conflict, Whiskey War, Denmark, Canada, Treaty, Flag, Liquor
###
Text: NASA launched the Discovery program to explore the solar system. It comprises a series of expeditions that have continued from the program's launch in the 1990s to the present day. In the course of the 16 expeditions launched so far, the Moon, Mars, Mercury and Venus, among others, have been explored. Unlike other space programs, the Discovery program places particular emphasis on cost efficiency, true to the motto: "faster, better, cheaper".
Keywords: Space program, NASA, Expedition, Cost efficiency, Moon, Mars, Mercury, Venus
###
Text: {text}
Keywords:""")   
    
    # 2. define the completion request 
    request = CompletionRequest(prompt=prompt, 
                                    maximum_tokens=32, 
                                    temperature=0, 
                                    stop_sequences=["\n"])
    
    # 3. send the request to the API
    response = client.complete(request=request, model="luminous-base")
    
    # 4. get the completion
    completion = response.completions[0].completion
    return completion

def get_keywords_langchain(text):
    # TODO Write a prompt that generates keywords for any text
    # Tipp: Use few-shot learning
    
    # 1. define the prompt
    prompt = f"""### Instructions: Extract keywords from the text. ### Input: Text:{text}\n### Response:"""
    
    # 2. define the parameters
    params = {
        "temperature": 0.5,
        "model": "luminous-base-control-beta",
        "maximum_tokens": 100
    }

    # 3. define the model
    aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), **params)
    
    # 4. get the completion
    response = aleph_alpha(prompt=prompt, stop=["\n"])
    return response
    

for text in texts:
    keywords = get_keywords(text)
    print(keywords)

 National sustainable development strategies, OECD, Agenda 21, Sustainable development, Policy integration, Intergenerational timeframe, Analysis and assessments, Co-ordination and institutions
 National sustainable development strategy, National strategy, Strategy, Implementation, Implementation, Implementation, Implementation, Implementation, Implementation, Implementation, Implementation, Implementation, Implementation, Implementation,
 National sustainable development strategy, National strategy, National sustainable development, National sustainable development strategy, National sustainable development strategy, National sustainable development strategy, National sustainable development strategy,
 National Sustainable Development Strategies, OECD, United Nations, Strategy, Policy integration, Inter-generational timeframe, Analysis and assessments, Co-ordination and institutions
 Sustainable development, National strategy, Environmental pillar, Social pillar, Trade-offs, Environme

In [13]:
print(texts[4])

 POLICY INTEGRATION

POLICY INTEGRATION – **15**

Sustainable development has been defined a variety of ways, but in practice it has come to mean development that achieves a balance among economic, environmental and social objectives for both present and future generations. The integration of the three dimensions of sustainable development is one of the most difficult balances to achieve in formulating a national strategy. In practice, most national strategies have a greater focus on environmental issues with some attempts to incorporate economic aspects. The social pillar has been the most neglected. As a result, few national strategies develop abilities for considering and making trade-offs among the three areas in overall policy-making.

Most countries emphasize environmental components in their sustainable development strategies. For example, the main focus of the Danish strategy is on integrating environmental considerations into a number of specific sectors. The Australian Strate

### Let's use Langchain to run a chain

In [14]:
# use a chain from langchain to generate a text

from langchain import PromptTemplate, LLMChain

template = """### Instructions: Solve the task based on the text below".

### Input:
{context}

### Task: {question}

### Response:"""

prompt = PromptTemplate(template=template, input_variables=["question", "context"])

llm = AlephAlpha(model="luminous-base-control-beta", maximum_tokens=32, stop_sequences=["###:"], aleph_alpha_api_key=os.getenv("AA_TOKEN"))

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Give me a list of countries that include social elements."

answer = llm_chain.run(question = question, context = texts[4])

print(answer)

 - Belgium
- Germany
- New Zealand
- Sweden


In [15]:
# TODO write two prompts one, that generates a question about a text and one that then answers the questions

def generate_questions_and_answers(text):
    # TODO Write a prompt that generates questions and answers for any text
    # Tipp: Use few-shot learning
        
    # 1. TODO define the prompt
    questions_prompt = Prompt.from_text(f"""### Instructions: Generate one specific question about the text below.
                                        
                                        ### Input: {text}
                                        
                                        ### Response:""")
    
    # 2. TODO define the completion request for the questions
    request = CompletionRequest(prompt=questions_prompt, temperature=0.0, stop_sequences=["\n"])
    response = client.complete(request=request, model="luminous-base-control-beta")
    question = response.completions[0].completion
    
    # 3. get the completion for answers
    answers_prompt = Prompt.from_text(f"""### Instructions: Generate an answer to the question below based on the text.
                                        
                                        ### Input: {text}
                                        
                                        ### Question: {question}
                                        
                                        ### Response:""")   
    request = CompletionRequest(prompt=answers_prompt, temperature=0.0, maximum_tokens=128)
    response = client.complete(request=request, model="luminous-base-control-beta")
        
    # 4. get the completion
    completion = response.completions[0].completion
    
    return question, completion

text = texts[9]

question, completion = generate_questions_and_answers(text)
print(f"Question: {question}")
print(f"Answer: \n{completion}")

Question:  What are some of the ways that stakeholders are involved in the development and implementation of national strategies for sustainable development?
Answer: 
 Some of the ways that stakeholders are involved in the development and implementation of national strategies for sustainable development include:
 
 - being consulted in the development of national strategies,
 - being involved in special commissions and councils which provide advice to but are separate from the government bodies which implement the strategy,
 - being included alongside government bodies as part of their overall co-ordination structure for sustainable development,



In [16]:
# TODO write a function where the user chats with an AI, store the conversation in a list as a memory
history = []
def chat_with_ai(message):
    # TODO Write a prompt that generates questions and answers for any text
    # Tipp: Use few-shot learning
    history.append(f"User: {message}")
    breaker = "\n"
    # 1. TODO define the prompt with the message and the history
    prompt = Prompt.from_text(f"""### Instructions: You are a friendly and helpful AI. You are chatting with a human user. Try your best to answer their questions.

### History: 
{breaker.join(history)}

### AI:""")
    
    # 2. TODO define the completion request for the answer
    request = CompletionRequest(prompt=prompt, temperature=0.5,)
    response = client.complete(request=request, model="luminous-base-control")
    answer = response.completions[0].completion
    
    
    
    history.append(f"AI: {answer}")
    
    return "\n".join(history)


In [20]:
print(chat_with_ai("What can you tell me about Heidelberg?"))

User: Maybe you can help me out. What are the most important things to know about boats?
AI:  Sure! Boats are a type of watercraft used for transportation, fishing, and recreation. They are typically powered by sail or motor, and can be made from wood, metal, plastic, or fiberglass. The size and type of boat will depend on the intended use. For example, a small dinghy is
User: How expensive are big boats?
AI:  Typically, bigger boats are more expensive than smaller ones. However, there are many factors that affect the cost, such as size, type, and features. For example, a luxury yacht can cost hundreds of thousands of dollars, while a small fishing boat might only cost a few thousand dollars.
User: Tell me more about the smaller ones
AI:  Smaller boats are often used for recreational purposes, such as fishing or swimming. They are generally less expensive than larger boats, but still provide plenty of features and amenities. For example, a fishing boat may have a small motor and a few 